In [142]:
import sys
import math
import time
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error
from scipy.stats import wasserstein_distance
import warnings
warnings.filterwarnings("ignore")

class SGLD:

    def __init__(self, beta, lam_c, lr, q, seed, real, epsilon):
        self.beta = beta # Temperature para
        self.lam_c = lam_c # Regularization
        self.lr = lr # Learning rate
        self.q = q
        self.seed = seed
        self.real = real
        self.epsilon = epsilon
        self.theta_star = real

    def func_excess(self, x, theta):
        
        return (self.q - 1*(x < theta)) * (x - theta)
    
    def cal_H(self, x, theta):
        H = - self.q + 1 * (x < theta) + 2 * self.lam_c * theta
        return H
        
    def estimate(self, x_array_sd1, x_array_sd2, x_array_sd3, x_array_sd4, x_array_sd5):
        np.random.seed(self.seed)
        theta_sd1 = np.random.normal(0, 1)
        theta_sd2 = np.random.normal(0, 1)
        theta_sd3 = np.random.normal(0, 1)
        theta_sd4 = np.random.normal(0, 1)
        theta_sd5 = np.random.normal(0, 1)# initial value \theta_0
        theta_dup_sd1 = [theta_sd1]; theta_dup_sd2 = [theta_sd2]; 
        theta_dup_sd3 = [theta_sd3]; theta_dup_sd4 = [theta_sd4]; 
        theta_dup_sd5 = [theta_sd5]; 
        real_value = [self.real for _ in range(1000)]
        
        # updating
        for i in range(len(x_array_sd1)):
            theta_sd1 += -self.lr * self.cal_H(x_array_sd1[i], theta_sd1) \
                     + math.sqrt((2 * self.lr / self.beta)) * np.random.normal(0, 1)
            theta_dup_sd1.append(theta_sd1)
            
            theta_sd2 += -self.lr * self.cal_H(x_array_sd2[i], theta_sd2) \
                     + math.sqrt((2 * self.lr / self.beta)) * np.random.normal(0, 1)
            theta_dup_sd2.append(theta_sd2)
            
            theta_sd3 += -self.lr * self.cal_H(x_array_sd3[i], theta_sd3) \
                     + math.sqrt((2 * self.lr / self.beta)) * np.random.normal(0, 1)
            theta_dup_sd3.append(theta_sd3)
            
            theta_sd4 += -self.lr * self.cal_H(x_array_sd4[i], theta_sd4) \
                     + math.sqrt((2 * self.lr / self.beta)) * np.random.normal(0, 1)
            theta_dup_sd4.append(theta_sd4)
            
            theta_sd5 += -self.lr * self.cal_H(x_array_sd5[i], theta_sd5) \
                     + math.sqrt((2 * self.lr / self.beta)) * np.random.normal(0, 1)
            theta_dup_sd5.append(theta_sd5)
            

            if i % 1001==0 and i > 1000:
                excess_risk_sd1 = np.mean(self.func_excess(x_array_sd1, theta = theta_sd1) - self.func_excess(x_array_sd1, theta = self.theta_star)) + lam_c * theta_sd1**2 - lam_c * self.theta_star**2
                excess_risk_sd2 = np.mean(self.func_excess(x_array_sd2, theta = theta_sd2) - self.func_excess(x_array_sd2, theta = self.theta_star)) + lam_c * theta_sd2**2 - lam_c * self.theta_star**2
                excess_risk_sd3 = np.mean(self.func_excess(x_array_sd3, theta = theta_sd3) - self.func_excess(x_array_sd3, theta = self.theta_star)) + lam_c * theta_sd3**2 - lam_c * self.theta_star**2
                excess_risk_sd4 = np.mean(self.func_excess(x_array_sd4, theta = theta_sd4) - self.func_excess(x_array_sd4, theta = self.theta_star)) + lam_c * theta_sd4**2 - lam_c * self.theta_star**2
                excess_risk_sd5 = np.mean(self.func_excess(x_array_sd5, theta = theta_sd5) - self.func_excess(x_array_sd5, theta = self.theta_star)) + lam_c * theta_sd5**2 - lam_c * self.theta_star**2
                excess_risk = excess_risk_sd1 + excess_risk_sd2 + excess_risk_sd3 + excess_risk_sd4 + excess_risk_sd5
                excess_risk = (excess_risk_sd1 + excess_risk_sd2 + excess_risk_sd3 + excess_risk_sd4 + excess_risk_sd5) / 5

                if np.abs(excess_risk) <= self.epsilon:
                    print(i)
                    return (theta_sd1 + theta_sd2 + theta_sd3 + theta_sd4 + theta_sd5) / 5, excess_risk
                
                
        return (theta_sd1 + theta_sd2 + theta_sd3 + theta_sd4 + theta_sd5) / 5, excess_risk #, expectation
    
################################################################################################################################################
class SGHMC:

    def __init__(self, beta, lam_c, lr, gamma, q, seed, real, epsilon):
        self.beta = beta # Temperature para
        self.lam_c = lam_c # Regularization
        self.lr = lr # Learning rate
        self.q = q
        self.gamma = gamma # momentum
        self.seed = seed
        self.real = real
        self.epsilon = epsilon
        self.theta_star = real
        
    def func_excess(self, x, theta):
        # theta = 2.944 is optimal value, quantile = 0.95
        return (self.q - 1*(x < theta)) * (x - theta)

    def cal_H(self, x, theta):
        """ m = 1"""
        H = -self.q + 1*(x < theta) + 2 * self.lam_c * theta
        return H

    def estimate(self, x_array_sd1, x_array_sd2, x_array_sd3, x_array_sd4, x_array_sd5):
        np.random.seed(self.seed)
        theta_sd1 = np.random.normal(0, 1)
        theta_sd2 = np.random.normal(0, 1)
        theta_sd3 = np.random.normal(0, 1)
        theta_sd4 = np.random.normal(0, 1)
        theta_sd5 = np.random.normal(0, 1)# initial value \theta_0
        vol_sd1 = 0
        vol_sd2 = 0
        vol_sd3 = 0
        vol_sd4 = 0
        vol_sd5 = 0
        theta_dup_sd1 = [theta_sd1]; theta_dup_sd2 = [theta_sd2]; 
        theta_dup_sd3 = [theta_sd3]; theta_dup_sd4 = [theta_sd4]; 
        theta_dup_sd5 = [theta_sd5]; 
        real_value = [self.real for _ in range(1000)]

        # updating
        for i in range(len(x_array_sd1)):
            vol_sd1 += - self.lr * self.gamma * vol_sd1 - self.lr * self.cal_H(x_array_sd1[i], theta_sd1) + math.sqrt((2 * self.lr * self.gamma / self.beta)) * np.random.normal(0, 1)
            theta_sd1 += self.lr * vol_sd1
            theta_dup_sd1.append(theta_sd1)
            
            vol_sd2 += - self.lr * self.gamma * vol_sd2 - self.lr * self.cal_H(x_array_sd2[i], theta_sd2) + math.sqrt((2 * self.lr * self.gamma / self.beta)) * np.random.normal(0, 1)
            theta_sd2 += self.lr * vol_sd2
            theta_dup_sd2.append(theta_sd2)
            
            vol_sd3 += - self.lr * self.gamma * vol_sd3 - self.lr * self.cal_H(x_array_sd3[i], theta_sd3) + math.sqrt((2 * self.lr * self.gamma / self.beta)) * np.random.normal(0, 1)
            theta_sd3 += self.lr * vol_sd3
            theta_dup_sd3.append(theta_sd3)
            
            vol_sd4 += - self.lr * self.gamma * vol_sd4 - self.lr * self.cal_H(x_array_sd4[i], theta_sd4) + math.sqrt((2 * self.lr * self.gamma / self.beta)) * np.random.normal(0, 1)
            theta_sd4 += self.lr * vol_sd4
            theta_dup_sd4.append(theta_sd4)
            
            vol_sd5 += - self.lr * self.gamma * vol_sd5 - self.lr * self.cal_H(x_array_sd5[i], theta_sd5) + math.sqrt((2 * self.lr * self.gamma / self.beta)) * np.random.normal(0, 1)
            theta_sd5 += self.lr * vol_sd5
            theta_dup_sd5.append(theta_sd5)
            
            
            if i % 1001==0 and i > 1000:
                excess_risk_sd1 = np.mean(self.func_excess(x_array_sd1, theta = theta_sd1)) + lam_c * theta_sd1**2 - np.mean(self.func_excess(x_array_sd1, theta = self.theta_star)) - lam_c * self.theta_star**2
                excess_risk_sd2 = np.mean(self.func_excess(x_array_sd2, theta = theta_sd2)) + lam_c * theta_sd2**2 - np.mean(self.func_excess(x_array_sd2, theta = self.theta_star)) - lam_c * self.theta_star**2
                excess_risk_sd3 = np.mean(self.func_excess(x_array_sd3, theta = theta_sd3)) + lam_c * theta_sd3**2 - np.mean(self.func_excess(x_array_sd3, theta = self.theta_star)) - lam_c * self.theta_star**2
                excess_risk_sd4 = np.mean(self.func_excess(x_array_sd4, theta = theta_sd4)) + lam_c * theta_sd4**2 - np.mean(self.func_excess(x_array_sd4, theta = self.theta_star)) - lam_c * self.theta_star**2
                excess_risk_sd5 = np.mean(self.func_excess(x_array_sd5, theta = theta_sd5)) + lam_c * theta_sd5**2 - np.mean(self.func_excess(x_array_sd5, theta = self.theta_star)) - lam_c * self.theta_star**2
                excess_risk = (excess_risk_sd1 + excess_risk_sd2 + excess_risk_sd3 + excess_risk_sd4 + excess_risk_sd5) / 5

                if np.abs(excess_risk) <= self.epsilon:
                    print(i)
                    return (theta_sd1 + theta_sd2 + theta_sd3 + theta_sd4 + theta_sd5) / 5, excess_risk


        return (theta_sd1 + theta_sd2 + theta_sd3 + theta_sd4 + theta_sd5), excess_risk #expectation

In [55]:
beta = 1e+8
lam_c = 1e-5
lr = 1e-3
gamma = 0.5 # gamma的选取对结果影响很大
seed_1= 1111; seed_2 = 2222; seed_3 = 3333; seed_4 = 4444; seed_5 = 5555

## Guassian dist.
### Guassian(-1, 1)  0.95

In [128]:
np.random.seed(1111)
x_norm_mu1_sig1_sd1 = np.random.normal(-1, 1, int(1e+5))
np.random.seed(2222)
x_norm_mu1_sig1_sd2 = np.random.normal(-1, 1, int(1e+5))
np.random.seed(3333)
x_norm_mu1_sig1_sd3 = np.random.normal(-1, 1, int(1e+5))
np.random.seed(4444)
x_norm_mu1_sig1_sd4 = np.random.normal(-1, 1, int(1e+5))
np.random.seed(5555)
x_norm_mu1_sig1_sd5 = np.random.normal(-1, 1, int(1e+5))

In [129]:
## SGLD
Model095_SGLD_sd1 = SGLD(beta, lam_c, lr, 0.95, seed_1, 0.645, 0.001)

start_time = time.time()
theta_SGLD, expected_excess_risk  = Model095_SGLD_sd1.estimate(x_norm_mu1_sig1_sd1, x_norm_mu1_sig1_sd2, x_norm_mu1_sig1_sd3, x_norm_mu1_sig1_sd4, x_norm_mu1_sig1_sd5)
end_time = time.time()

print(f'Expected excess risk of SGLD is {expected_excess_risk}')
print(f'Value of SGLD is {theta_SGLD}')
print(f'耗时: {(end_time - start_time) / 5}秒')

18018
Expected excess risk of SGLD is 0.0009397854279160938
Value of SGLD is 0.6386796608501386
耗时: 0.11100606918334961秒


In [130]:
## SGHMC

Model095_SGHMC_sd1 = SGHMC(beta, lam_c, lr, gamma, 0.95, seed_1, 0.645, 0.001)
start_time = time.time()
theta_SGHMC, expected_excess_risk  = Model095_SGHMC_sd1.estimate(x_norm_mu1_sig1_sd1, x_norm_mu1_sig1_sd2, x_norm_mu1_sig1_sd3, x_norm_mu1_sig1_sd4, x_norm_mu1_sig1_sd5)
end_time = time.time()

print(f'excess_risk of SGHMC is {expected_excess_risk}')
print(f'Value of SGHMC is {theta_SGHMC}')
print(f'耗时: {(end_time - start_time) / 5}秒')

12012
excess_risk of SGHMC is 0.0009700996012607281
Value of SGHMC is 0.7789031440875854
耗时: 0.07880601882934571秒


### Guassian(-1, 1)  0.99

In [131]:
## SGLD

Model099_SGLD_sd1 = SGLD(beta, lam_c, lr, 0.99, seed_1, 1.326, 0.0001)
#Model099_SGLD_sd2 = SGLD(beta, lam_c, lr, 0.99, seed_2, 1.326, 0.01)
#Model099_SGLD_sd3 = SGLD(beta, lam_c, lr, 0.99, seed_3, 1.326, 0.01)
#Model099_SGLD_sd4 = SGLD(beta, lam_c, lr, 0.99, seed_4, 1.326, 0.01)
#Model099_SGLD_sd5 = SGLD(beta, lam_c, lr, 0.99, seed_5, 1.326, 0.01)

start_time = time.time()
theta_SGLD, expected_excess_risk  = Model099_SGLD_sd1.estimate(x_norm_mu1_sig1_sd1, x_norm_mu1_sig1_sd2, x_norm_mu1_sig1_sd3, x_norm_mu1_sig1_sd4, x_norm_mu1_sig1_sd5)
#theta_SGLD_sd2, theta_all_SGLD_sd2_95,SGLD_sd2_95_wd  = Model095_SGLD_sd2.estimate(x_norm_mu1_sig1_sd2)
#theta_SGLD_sd3, theta_all_SGLD_sd3_95,SGLD_sd3_95_wd  = Model095_SGLD_sd3.estimate(x_norm_mu1_sig1_sd3)
#theta_SGLD_sd4, theta_all_SGLD_sd4_95,SGLD_sd4_95_wd  = Model095_SGLD_sd4.estimate(x_norm_mu1_sig1_sd4)
#theta_SGLD_sd5, theta_all_SGLD_sd5_95,SGLD_sd5_95_wd  = Model095_SGLD_sd5.estimate(x_norm_mu1_sig1_sd5)
end_time = time.time()

print(f'Expected excess risk of SGLD is {expected_excess_risk}')
print(f'Value of SGLD is {theta_SGLD}')
print(f'耗时: {(end_time - start_time) / 5}秒')

62062
Expected excess risk of SGLD is 9.419428703303796e-05
Value of SGLD is 1.2651763748189033
耗时: 0.37160658836364746秒


In [132]:
## SGHMC

Model099_SGHMC_sd1 = SGHMC(beta, lam_c, lr, gamma, 0.99, seed_1, 1.326, 0.0001)


start_time = time.time()
theta_SGHMC, expected_excess_risk  = Model099_SGHMC_sd1.estimate(x_norm_mu1_sig1_sd1, x_norm_mu1_sig1_sd2, x_norm_mu1_sig1_sd3, x_norm_mu1_sig1_sd4, x_norm_mu1_sig1_sd5)
end_time = time.time()

print(f'excess_risk of SGHMC is {expected_excess_risk}')
print(f'Value of SGHMC is {theta_SGHMC}')
print(f'耗时: {(end_time - start_time) / 5}秒')

13013
excess_risk of SGHMC is 9.889894222152003e-05
Value of SGHMC is 1.310529205444929
耗时: 0.07760701179504395秒


### Guassian(1, 2)  0.95

In [134]:
np.random.seed(1111)
x_norm_mu1_sig2_sd1 = np.random.normal(1, 2, int(1e+6))
np.random.seed(2222)
x_norm_mu1_sig2_sd2 = np.random.normal(1, 2, int(1e+6))
np.random.seed(3333)
x_norm_mu1_sig2_sd3 = np.random.normal(1, 2, int(1e+6))
np.random.seed(4444)
x_norm_mu1_sig2_sd4 = np.random.normal(1, 2, int(1e+6))
np.random.seed(5555)
x_norm_mu1_sig2_sd5 = np.random.normal(1, 2, int(1e+6))

In [135]:
## SGLD

Model095_SGLD_sd1 = SGLD(beta, lam_c, lr, 0.95, seed_1, 4.290, 0.0001)

start_time = time.time()
theta_SGLD, expected_excess_risk  = Model095_SGLD_sd1.estimate(x_norm_mu1_sig2_sd1, x_norm_mu1_sig2_sd2, x_norm_mu1_sig2_sd3, x_norm_mu1_sig2_sd4, x_norm_mu1_sig2_sd5)
end_time = time.time()

print(f'Expected excess risk of SGLD is {expected_excess_risk}')
print(f'Value of SGLD is {theta_SGLD}')
print(f'耗时: {(end_time - start_time) / 5}秒')

59059
Expected excess risk of SGLD is 8.434848526421e-05
Value of SGLD is 4.235385977685304
耗时: 1.8438060283660889秒


In [136]:
## SGHMC

Model095_SGHMC_sd1 = SGHMC(beta, lam_c, lr, gamma, 0.95, seed_1, 4.290, 0.0001)

start_time = time.time()
theta_SGHMC, expected_excess_risk  = Model095_SGHMC_sd1.estimate(x_norm_mu1_sig2_sd1, x_norm_mu1_sig2_sd2, x_norm_mu1_sig2_sd3, x_norm_mu1_sig2_sd4, x_norm_mu1_sig2_sd5)
end_time = time.time()

print(f'excess_risk of SGHMC is {expected_excess_risk}')
print(f'Value of SGHMC is {theta_SGHMC}')
print(f'耗时: {(end_time - start_time) / 5}秒')

23023
excess_risk of SGHMC is 9.820456612330696e-05
Value of SGHMC is 4.329333466384593
耗时: 0.6949999332427979秒


### Guassian(1, 2)  0.99

In [139]:
## SGLD

Model099_SGLD_sd1 = SGLD(beta, lam_c, lr, 0.99, seed_1, 5.653, 0.0001)

start_time = time.time()
theta_SGLD, expected_excess_risk = Model099_SGLD_sd1.estimate(x_norm_mu1_sig2_sd1, x_norm_mu1_sig2_sd2, x_norm_mu1_sig2_sd3, x_norm_mu1_sig2_sd4, x_norm_mu1_sig2_sd5)
end_time = time.time()

print(f'Expected excess risk of SGLD is {expected_excess_risk}')
print(f'Value of SGLD is {theta_SGLD}')
print(f'耗时: {(end_time - start_time) / 5}秒')

159159
Expected excess risk of SGLD is 9.983225028832184e-05
Value of SGLD is 5.524798918138304
耗时: 5.011406755447387秒


In [143]:
## SGHMC

Model099_SGHMC_sd1 = SGHMC(beta, lam_c, lr, gamma, 0.99, seed_1, 5.653, 0.0001)

start_time = time.time()
theta_SGHMC, expected_excess_risk  = Model099_SGHMC_sd1.estimate(x_norm_mu1_sig2_sd1, x_norm_mu1_sig2_sd2, x_norm_mu1_sig2_sd3, x_norm_mu1_sig2_sd4, x_norm_mu1_sig2_sd5)
end_time = time.time()

print(f'excess_risk of SGHMC is {expected_excess_risk}')
print(f'Value of SGHMC is {theta_SGHMC}')
print(f'耗时: {(end_time - start_time) / 5}秒')

62062
excess_risk of SGHMC is 9.642299506884607e-05
Value of SGHMC is 5.529168168137042
耗时: 1.830399990081787秒


### Guassian(3, 5)  0.95

In [145]:
np.random.seed(1111)
x_norm_mu3_sig5_sd1 = np.random.normal(3, 5, int(1e+6))
np.random.seed(2222)
x_norm_mu3_sig5_sd2 = np.random.normal(3, 5, int(1e+6))
np.random.seed(3333)
x_norm_mu3_sig5_sd3 = np.random.normal(3, 5, int(1e+6))
np.random.seed(4444)
x_norm_mu3_sig5_sd4 = np.random.normal(3, 5, int(1e+6))
np.random.seed(5555)
x_norm_mu3_sig5_sd5 = np.random.normal(3, 5, int(1e+6))

In [146]:
## SGLD

Model095_SGLD_sd1 = SGLD(beta, lam_c, lr, 0.95, seed_1, 11.224, 0.0001)
start_time = time.time()
theta_SGLD, expected_excess_risk  = Model095_SGLD_sd1.estimate(x_norm_mu3_sig5_sd1, x_norm_mu3_sig5_sd2, x_norm_mu3_sig5_sd3, x_norm_mu3_sig5_sd4, x_norm_mu3_sig5_sd5)
end_time = time.time()

print(f'Expected excess risk of SGLD is {expected_excess_risk}')
print(f'Value of SGLD is {theta_SGLD}')
print(f'耗时: {(end_time - start_time) / 5}秒')

169169
Expected excess risk of SGLD is 9.462153585307418e-05
Value of SGLD is 11.126626081864849
耗时: 5.315605640411377秒


In [148]:
## SGHMC

Model095_SGHMC_sd1 = SGHMC(beta, lam_c, lr, gamma, 0.95, seed_1, 11.224, 0.0001)

start_time = time.time()
theta_SGHMC, expected_excess_risk  = Model095_SGHMC_sd1.estimate(x_norm_mu3_sig5_sd1, x_norm_mu3_sig5_sd2, x_norm_mu3_sig5_sd3, x_norm_mu3_sig5_sd4, x_norm_mu3_sig5_sd5)
end_time = time.time()

print(f'excess_risk of SGHMC is {expected_excess_risk}')
print(f'Value of SGHMC is {theta_SGHMC}')
print(f'耗时: {(end_time - start_time) / 5}秒')

69069
excess_risk of SGHMC is 9.882561450861339e-05
Value of SGHMC is 11.136921103457118
耗时: 2.0291982173919676秒


### Guassian(3, 5)  0.99

In [149]:
## SGLD

Model099_SGLD_sd1 = SGLD(beta, lam_c, lr, 0.99, seed_1, 14.632, 0.0001)

start_time = time.time()
theta_SGLD, expected_excess_risk  = Model099_SGLD_sd1.estimate(x_norm_mu3_sig5_sd1, x_norm_mu3_sig5_sd2, x_norm_mu3_sig5_sd3, x_norm_mu3_sig5_sd4, x_norm_mu3_sig5_sd5)
end_time = time.time()

print(f'Expected excess risk of SGLD is {expected_excess_risk}')
print(f'Value of SGLD is {theta_SGLD}')
print(f'耗时: {(end_time - start_time) / 5}秒')

488488
Expected excess risk of SGLD is 9.853985763201307e-05
Value of SGLD is 14.379150165509822
耗时: 15.769285869598388秒


In [150]:
## SGHMC

Model099_SGHMC_sd1 = SGHMC(beta, lam_c, lr, gamma, 0.99, seed_1, 14.632, 0.0001)

start_time = time.time()
theta_SGHMC, expected_excess_risk  = Model099_SGHMC_sd1.estimate(x_norm_mu3_sig5_sd1, x_norm_mu3_sig5_sd2, x_norm_mu3_sig5_sd3, x_norm_mu3_sig5_sd4, x_norm_mu3_sig5_sd5)
end_time = time.time()

print(f'excess_risk of SGHMC is {expected_excess_risk}')
print(f'Value of SGHMC is {theta_SGHMC}')
print(f'耗时: {(end_time - start_time) / 5}秒')

218218
excess_risk of SGHMC is 9.916638881069902e-05
Value of SGHMC is 14.378597427070634
耗时: 6.828199577331543秒


## Logistic dist.
### Logistic(0, 1)  0.95

In [152]:
np.random.seed(1111)
x_logistic_mu0_scale1_sd1 = np.random.logistic(0, 1, int(1e+6))
np.random.seed(2222)
x_logistic_mu0_scale1_sd2 = np.random.logistic(0, 1, int(1e+6))
np.random.seed(3333)
x_logistic_mu0_scale1_sd3 = np.random.logistic(0, 1, int(1e+6))
np.random.seed(4444)
x_logistic_mu0_scale1_sd4 = np.random.logistic(0, 1, int(1e+6))
np.random.seed(5555)
x_logistic_mu0_scale1_sd5 = np.random.logistic(0, 1, int(1e+6))

In [153]:
## SGLD

Model095_SGLD_sd1 = SGLD(beta, lam_c, lr, 0.95, seed_1, 2.944, 0.0001)

start_time = time.time()
theta_SGLD, expected_excess_risk  = Model095_SGLD_sd1.estimate(x_logistic_mu0_scale1_sd1, x_logistic_mu0_scale1_sd2, x_logistic_mu0_scale1_sd3, x_logistic_mu0_scale1_sd4, x_logistic_mu0_scale1_sd5)
end_time = time.time()

print(f'Expected excess risk of SGLD is {expected_excess_risk}')
print(f'Value of SGLD is {theta_SGLD}')
print(f'耗时: {(end_time - start_time) / 5}秒')

59059
Expected excess risk of SGLD is 9.489021803808454e-05
Value of SGLD is 2.883165084161937
耗时: 1.8171990871429444秒


In [154]:
## SGHMC

Model095_SGHMC_sd1 = SGHMC(beta, lam_c, lr, gamma, 0.95, seed_1, 2.944, 0.0001)

start_time = time.time()
theta_SGHMC, expected_excess_risk  = Model095_SGHMC_sd1.estimate(x_logistic_mu0_scale1_sd1, x_logistic_mu0_scale1_sd2, x_logistic_mu0_scale1_sd3, x_logistic_mu0_scale1_sd4, x_logistic_mu0_scale1_sd5)
end_time = time.time()

print(f'excess_risk of SGHMC is {expected_excess_risk}')
print(f'Value of SGHMC is {theta_SGHMC}')
print(f'耗时: {(end_time - start_time) / 5}秒')

20020
excess_risk of SGHMC is 8.898271030448733e-05
Value of SGHMC is 2.968912140875896
耗时: 0.5841999053955078秒


### Logistic(0, 1)  0.99

In [155]:
## SGLD

Model099_SGLD_sd1 = SGLD(beta, lam_c, lr, 0.99, seed_1, 4.595, 0.0001)

start_time = time.time()
theta_SGLD, expected_excess_risk  = Model099_SGLD_sd1.estimate(x_logistic_mu0_scale1_sd1, x_logistic_mu0_scale1_sd2, x_logistic_mu0_scale1_sd3, x_logistic_mu0_scale1_sd4, x_logistic_mu0_scale1_sd5)
end_time = time.time()

print(f'Expected excess risk of SGLD is {expected_excess_risk}')
print(f'Value of SGLD is {theta_SGLD}')
print(f'耗时: {(end_time - start_time) / 5}秒')

195195
Expected excess risk of SGLD is 9.339227850100449e-05
Value of SGLD is 4.454960724303989
耗时: 6.134206295013428秒


In [156]:
## SGHMC

Model099_SGHMC_sd1 = SGHMC(beta, lam_c, lr, gamma, 0.99, seed_1, 4.595, 0.0001)

start_time = time.time()
theta_SGHMC, expected_excess_risk  = Model099_SGHMC_sd1.estimate(x_logistic_mu0_scale1_sd1, x_logistic_mu0_scale1_sd2, x_logistic_mu0_scale1_sd3, x_logistic_mu0_scale1_sd4, x_logistic_mu0_scale1_sd5)
end_time = time.time()

print(f'excess_risk of SGHMC is {expected_excess_risk}')
print(f'Value of SGHMC is {theta_SGHMC}')
print(f'耗时: {(end_time - start_time) / 5}秒')

83083
excess_risk of SGHMC is 9.962969136508831e-05
Value of SGHMC is 4.453875936747583
耗时: 2.4972061634063722秒


### Logistic(-1, 1)  0.95

In [157]:
np.random.seed(1111)
x_logistic_mu1_scale1_sd1 = np.random.logistic(-1, 1, int(1e+6))
np.random.seed(2222)
x_logistic_mu1_scale1_sd2 = np.random.logistic(-1, 1, int(1e+6))
np.random.seed(3333)
x_logistic_mu1_scale1_sd3 = np.random.logistic(-1, 1, int(1e+6))
np.random.seed(4444)
x_logistic_mu1_scale1_sd4 = np.random.logistic(-1, 1, int(1e+6))
np.random.seed(5555)
x_logistic_mu1_scale1_sd5 = np.random.logistic(-1, 1, int(1e+6))

In [158]:
## SGLD

Model095_SGLD_sd1 = SGLD(beta, lam_c, lr, 0.95, seed_1, 1.944, 0.0001)

start_time = time.time()
theta_SGLD, expected_excess_risk  = Model095_SGLD_sd1.estimate(x_logistic_mu1_scale1_sd1, x_logistic_mu1_scale1_sd2, x_logistic_mu1_scale1_sd3, x_logistic_mu1_scale1_sd4, x_logistic_mu1_scale1_sd5)
end_time = time.time()

print(f'Expected excess risk of SGLD is {expected_excess_risk}')
print(f'Value of SGLD is {theta_SGLD}')
print(f'耗时: {(end_time - start_time) / 5}秒')

56056
Expected excess risk of SGLD is 8.714227506768406e-05
Value of SGLD is 1.8859655955876857
耗时: 1.7655994415283203秒


In [159]:
## SGHMC

Model095_SGHMC_sd1 = SGHMC(beta, lam_c, lr, gamma, 0.95, seed_1, 1.944, 0.0001)

start_time = time.time()
theta_SGHMC, expected_excess_risk  = Model095_SGHMC_sd1.estimate(x_logistic_mu1_scale1_sd1, x_logistic_mu1_scale1_sd2, x_logistic_mu1_scale1_sd3, x_logistic_mu1_scale1_sd4, x_logistic_mu1_scale1_sd5)
end_time = time.time()

print(f'excess_risk of SGHMC is {expected_excess_risk}')
print(f'Value of SGHMC is {theta_SGHMC}')
print(f'耗时: {(end_time - start_time) / 5}秒')

15015
excess_risk of SGHMC is 8.496779358425094e-05
Value of SGHMC is 1.9070429078476927
耗时: 0.4317995548248291秒


### Logistic(-1, 1)  0.99

In [161]:
## SGLD

Model099_SGLD_sd1 = SGLD(beta, lam_c, lr, 0.99, seed_1, 3.595, 0.0001)

start_time = time.time()
theta_SGLD, expected_excess_risk  = Model099_SGLD_sd1.estimate(x_logistic_mu1_scale1_sd1, x_logistic_mu1_scale1_sd2, x_logistic_mu1_scale1_sd3, x_logistic_mu1_scale1_sd4, x_logistic_mu1_scale1_sd5)
end_time = time.time()

print(f'Expected excess risk of SGLD is {expected_excess_risk}')
print(f'Value of SGLD is {theta_SGLD}')
print(f'耗时: {(end_time - start_time) / 5}秒')

193193
Expected excess risk of SGLD is 9.718748867517841e-05
Value of SGLD is 3.4549642066729307
耗时: 6.180034446716308秒


In [162]:
## SGHMC

Model099_SGHMC_sd1 = SGHMC(beta, lam_c, lr, gamma, 0.99, seed_1, 3.595, 0.0001)
start_time = time.time()
theta_SGHMC, expected_excess_risk  = Model099_SGHMC_sd1.estimate(x_logistic_mu1_scale1_sd1, x_logistic_mu1_scale1_sd2, x_logistic_mu1_scale1_sd3, x_logistic_mu1_scale1_sd4, x_logistic_mu1_scale1_sd5)
end_time = time.time()

print(f'excess_risk of SGHMC is {expected_excess_risk}')
print(f'Value of SGHMC is {theta_SGHMC}')
print(f'耗时: {(end_time - start_time) / 5}秒')

87087
excess_risk of SGHMC is 9.877743647557059e-05
Value of SGHMC is 3.453585627858871
耗时: 2.6152059555053713秒


### Logistic(-3, 3)  0.95

In [163]:
np.random.seed(1111)
x_logistic_mu3_scale3_sd1 = np.random.logistic(-3, 3, int(1e+7))
np.random.seed(2222)
x_logistic_mu3_scale3_sd2 = np.random.logistic(-3, 3, int(1e+7))
np.random.seed(3333)
x_logistic_mu3_scale3_sd3 = np.random.logistic(-3, 3, int(1e+7))
np.random.seed(4444)
x_logistic_mu3_scale3_sd4 = np.random.logistic(-3, 3, int(1e+7))
np.random.seed(5555)
x_logistic_mu3_scale3_sd5 = np.random.logistic(-3, 3, int(1e+7))

In [164]:
## SGLD

Model095_SGLD_sd1 = SGLD(beta, lam_c, lr, 0.95, seed_1, 5.833, 0.0001)

start_time = time.time()
theta_SGLD, expected_excess_risk  = Model095_SGLD_sd1.estimate(x_logistic_mu3_scale3_sd1, x_logistic_mu3_scale3_sd2, x_logistic_mu3_scale3_sd3, x_logistic_mu3_scale3_sd4, x_logistic_mu3_scale3_sd5)
end_time = time.time()

print(f'Expected excess risk of SGLD is {expected_excess_risk}')
print(f'Value of SGLD is {theta_SGLD}')
print(f'耗时: {(end_time - start_time) / 5}秒')

191191
Expected excess risk of SGLD is 9.57186705436027e-05
Value of SGLD is 5.7165322952158295
耗时: 56.743565464019774秒


In [165]:
## SGHMC

Model095_SGHMC_sd1 = SGHMC(beta, lam_c, lr, gamma, 0.95, seed_1, 5.833, 0.0001)
start_time = time.time()
theta_SGHMC, expected_excess_risk  = Model095_SGHMC_sd1.estimate(x_logistic_mu3_scale3_sd1, x_logistic_mu3_scale3_sd2, x_logistic_mu3_scale3_sd3, x_logistic_mu3_scale3_sd4, x_logistic_mu3_scale3_sd5)
end_time = time.time()

print(f'excess_risk of SGHMC is {expected_excess_risk}')
print(f'Value of SGHMC is {theta_SGHMC}')
print(f'耗时: {(end_time - start_time) / 5}秒')

101101
excess_risk of SGHMC is 9.878714312253592e-05
Value of SGHMC is 5.74238022600843
耗时: 26.656200075149535秒


### Logistic(-3, 3)  0.99

In [166]:
## SGLD

Model099_SGLD_sd1 = SGLD(beta, lam_c, lr, 0.99, seed_1, 10.785, 0.0001)

start_time = time.time()
theta_SGLD, expected_excess_risk  = Model099_SGLD_sd1.estimate(x_logistic_mu3_scale3_sd1, x_logistic_mu3_scale3_sd2, x_logistic_mu3_scale3_sd3, x_logistic_mu3_scale3_sd4, x_logistic_mu3_scale3_sd5)
end_time = time.time()

print(f'Expected excess risk of SGLD is {expected_excess_risk}')
print(f'Value of SGLD is {theta_SGLD}')
print(f'耗时: {(end_time - start_time) / 5}秒')

746746
Expected excess risk of SGLD is 9.75842472870039e-05
Value of SGLD is 10.466053939675938
耗时: 210.6366229057312秒


In [167]:
## SGHMC

Model099_SGHMC_sd1 = SGHMC(beta, lam_c, lr, gamma, 0.99, seed_1, 10.785, 0.0001)
start_time = time.time()
theta_SGHMC, expected_excess_risk  = Model099_SGHMC_sd1.estimate(x_logistic_mu3_scale3_sd1, x_logistic_mu3_scale3_sd2, x_logistic_mu3_scale3_sd3, x_logistic_mu3_scale3_sd4, x_logistic_mu3_scale3_sd5)
end_time = time.time()

print(f'excess_risk of SGHMC is {expected_excess_risk}')
print(f'Value of SGHMC is {theta_SGHMC}')
print(f'耗时: {(end_time - start_time) / 5}秒')

340340
excess_risk of SGHMC is 9.929956913864364e-05
Value of SGHMC is 10.463444649700591
耗时: 86.56820111274719秒


## Gumbel dist.
### Gumbel (0, 1) 0.95

In [168]:
np.random.seed(1111)
x_g_0_1_sd1 = np.random.gumbel(loc=0.0, scale=1, size=int(1e+6))
np.random.seed(2222)
x_g_0_1_sd2 = np.random.gumbel(loc=0.0, scale=1, size=int(1e+6))
np.random.seed(3333)
x_g_0_1_sd3 = np.random.gumbel(loc=0.0, scale=1, size=int(1e+6))
np.random.seed(4444)
x_g_0_1_sd4 = np.random.gumbel(loc=0.0, scale=1, size=int(1e+6))
np.random.seed(5555)
x_g_0_1_sd5 = np.random.gumbel(loc=0.0, scale=1, size=int(1e+6))

In [169]:
## SGLD

Model095_SGLD_sd1 = SGLD(beta, lam_c, lr, 0.95, seed_1, 2.970, 0.0001)
start_time = time.time()
theta_SGLD, expected_excess_risk  = Model095_SGLD_sd1.estimate(x_g_0_1_sd1, x_g_0_1_sd2, x_g_0_1_sd3, x_g_0_1_sd4, x_g_0_1_sd5)
end_time = time.time()

print(f'Expected excess risk of SGLD is {expected_excess_risk}')
print(f'Value of SGLD is {theta_SGLD}')
print(f'耗时: {(end_time - start_time) / 5}秒')

64064
Expected excess risk of SGLD is 8.855077098582798e-05
Value of SGLD is 2.9120065373689776
耗时: 2.047999048233032秒


In [170]:
## SGHMC

Model095_SGHMC_sd1 = SGHMC(beta, lam_c, lr, gamma, 0.95, seed_1, 2.970, 0.0001)
start_time = time.time()
theta_SGHMC, expected_excess_risk  = Model095_SGHMC_sd1.estimate(x_g_0_1_sd1, x_g_0_1_sd2, x_g_0_1_sd3, x_g_0_1_sd4, x_g_0_1_sd5)
end_time = time.time()

print(f'excess_risk of SGHMC is {expected_excess_risk}')
print(f'Value of SGHMC is {theta_SGHMC}')
print(f'耗时: {(end_time - start_time) / 5}秒')

25025
excess_risk of SGHMC is 9.149774026205335e-05
Value of SGHMC is 3.0054941077394934
耗时: 0.8146055698394775秒


### Gumbel (0, 1) 0.99

In [171]:
## SGLD

Model099_SGLD_sd1 = SGLD(beta, lam_c, lr, 0.99, seed_1, 4.600, 0.0001)
start_time = time.time()
theta_SGLD, expected_excess_risk  = Model099_SGLD_sd1.estimate(x_g_0_1_sd1, x_g_0_1_sd2, x_g_0_1_sd3, x_g_0_1_sd4, x_g_0_1_sd5)
end_time = time.time()

print(f'Expected excess risk of SGLD is {expected_excess_risk}')
print(f'Value of SGLD is {theta_SGLD}')
print(f'耗时: {(end_time - start_time) / 5}秒')

198198
Expected excess risk of SGLD is 9.930719362720597e-05
Value of SGLD is 4.450715017399689
耗时: 6.233599615097046秒


In [172]:
## SGHMC

Model099_SGHMC_sd1 = SGHMC(beta, lam_c, lr, gamma, 0.99, seed_1, 4.600, 0.0001)
start_time = time.time()
theta_SGHMC, expected_excess_risk  = Model099_SGHMC_sd1.estimate(x_g_0_1_sd1, x_g_0_1_sd2, x_g_0_1_sd3, x_g_0_1_sd4, x_g_0_1_sd5)
end_time = time.time()

print(f'excess_risk of SGHMC is {expected_excess_risk}')
print(f'Value of SGHMC is {theta_SGHMC}')
print(f'耗时: {(end_time - start_time) / 5}秒')

90090
excess_risk of SGHMC is 9.649050222153356e-05
Value of SGHMC is 4.456086675928783
耗时: 2.690205383300781秒


### Gumbel (0, 2) 0.95

In [173]:
np.random.seed(1111)
x_g_0_2_sd1 = np.random.gumbel(loc=0.0, scale=2, size=int(1e+7))
np.random.seed(2222)
x_g_0_2_sd2 = np.random.gumbel(loc=0.0, scale=2, size=int(1e+7))
np.random.seed(3333)
x_g_0_2_sd3 = np.random.gumbel(loc=0.0, scale=2, size=int(1e+7))
np.random.seed(4444)
x_g_0_2_sd4 = np.random.gumbel(loc=0.0, scale=2, size=int(1e+7))
np.random.seed(5555)
x_g_0_2_sd5 = np.random.gumbel(loc=0.0, scale=2, size=int(1e+7))

In [174]:
## SGLD

Model095_SGLD_sd1 = SGLD(beta, lam_c, lr, 0.95, seed_1, 5.940, 0.0001)
start_time = time.time()
theta_SGLD, expected_excess_risk  = Model095_SGLD_sd1.estimate(x_g_0_2_sd1, x_g_0_2_sd2, x_g_0_2_sd3, x_g_0_2_sd4, x_g_0_2_sd5)
end_time = time.time()

print(f'Expected excess risk of SGLD is {expected_excess_risk}')
print(f'Value of SGLD is {theta_SGLD}')
print(f'耗时: {(end_time - start_time) / 5}秒')

129129
Expected excess risk of SGLD is 9.504701756513813e-05
Value of SGLD is 5.850097851620015
耗时: 37.832629108428954秒


In [175]:
## SGHMC

Model095_SGHMC_sd1 = SGHMC(beta, lam_c, lr, gamma, 0.95, seed_1, 5.940, 0.0001)
start_time = time.time()
theta_SGHMC, expected_excess_risk  = Model095_SGHMC_sd1.estimate(x_g_0_2_sd1, x_g_0_2_sd2, x_g_0_2_sd3, x_g_0_2_sd4, x_g_0_2_sd5)
end_time = time.time()

print(f'excess_risk of SGHMC is {expected_excess_risk}')
print(f'Value of SGHMC is {theta_SGHMC}')
print(f'耗时: {(end_time - start_time) / 5}秒')

66066
excess_risk of SGHMC is 8.985489667545716e-05
Value of SGHMC is 5.858709100219052
耗时: 18.831241178512574秒


### Gumbel (0, 2) 0.99

In [176]:
## SGLD

Model099_SGLD_sd1 = SGLD(beta, lam_c, lr, 0.99, seed_1, 9.200, 0.0001)
start_time = time.time()
theta_SGLD, expected_excess_risk  = Model099_SGLD_sd1.estimate(x_g_0_2_sd1, x_g_0_2_sd2, x_g_0_2_sd3, x_g_0_2_sd4, x_g_0_2_sd5)
end_time = time.time()

print(f'Expected excess risk of SGLD is {expected_excess_risk}')
print(f'Value of SGLD is {theta_SGLD}')
print(f'耗时: {(end_time - start_time) / 5}秒')

476476
Expected excess risk of SGLD is 9.698329259395075e-05
Value of SGLD is 8.970686482731278
耗时: 136.09199919700623秒


In [177]:
## SGHMC

Model099_SGHMC_sd1 = SGHMC(beta, lam_c, lr, gamma, 0.99, seed_1, 9.200, 0.0001)
start_time = time.time()
theta_SGHMC, expected_excess_risk  = Model099_SGHMC_sd1.estimate(x_g_0_2_sd1, x_g_0_2_sd2, x_g_0_2_sd3, x_g_0_2_sd4, x_g_0_2_sd5)
end_time = time.time()

print(f'excess_risk of SGHMC is {expected_excess_risk}')
print(f'Value of SGHMC is {theta_SGHMC}')
print(f'耗时: {(end_time - start_time) / 5}秒')

219219
excess_risk of SGHMC is 9.927403949452902e-05
Value of SGHMC is 8.969335728175116
耗时: 53.86119985580444秒


### Gumbel (1, 2) 0.95

In [178]:
np.random.seed(1111)
x_g_1_2_sd1 = np.random.gumbel(loc=1.0, scale=2, size=int(1e+7))
np.random.seed(2222)
x_g_1_2_sd2 = np.random.gumbel(loc=1.0, scale=2, size=int(1e+7))
np.random.seed(3333)
x_g_1_2_sd3 = np.random.gumbel(loc=1.0, scale=2, size=int(1e+7))
np.random.seed(4444)
x_g_1_2_sd4 = np.random.gumbel(loc=1.0, scale=2, size=int(1e+7))
np.random.seed(5555)
x_g_1_2_sd5 = np.random.gumbel(loc=1.0, scale=2, size=int(1e+7))

In [179]:
## SGLD

Model095_SGLD_sd1 = SGLD(beta, lam_c, lr, 0.95, seed_1, 6.940, 0.0001)
start_time = time.time()
theta_SGLD, expected_excess_risk  = Model095_SGLD_sd1.estimate(x_g_1_2_sd1, x_g_1_2_sd2, x_g_1_2_sd3, x_g_1_2_sd4, x_g_1_2_sd5)
end_time = time.time()

print(f'Expected excess risk of SGLD is {expected_excess_risk}')
print(f'Value of SGLD is {theta_SGLD}')
print(f'耗时: {(end_time - start_time) / 5}秒')

133133
Expected excess risk of SGLD is 8.793220097024248e-05
Value of SGLD is 6.850825054133732
耗时: 36.88344020843506秒


In [180]:
## SGHMC

Model095_SGHMC_sd1 = SGHMC(beta, lam_c, lr, gamma, 0.95, seed_1, 6.940, 0.0001)
start_time = time.time()
theta_SGHMC, expected_excess_risk  = Model095_SGHMC_sd1.estimate(x_g_1_2_sd1, x_g_1_2_sd2, x_g_1_2_sd3, x_g_1_2_sd4, x_g_1_2_sd5)
end_time = time.time()

print(f'excess_risk of SGHMC is {expected_excess_risk}')
print(f'Value of SGHMC is {theta_SGHMC}')
print(f'耗时: {(end_time - start_time) / 5}秒')

66066
excess_risk of SGHMC is 8.687360603261883e-05
Value of SGHMC is 6.859056664038536
耗时: 16.80640697479248秒


### Gumbel (1, 2) 0.99

In [181]:
## SGLD

Model099_SGLD_sd1 = SGLD(beta, lam_c, lr, 0.99, seed_1, 10.200, 0.0001)
start_time = time.time()
theta_SGLD, expected_excess_risk  = Model099_SGLD_sd1.estimate(x_g_1_2_sd1, x_g_1_2_sd2, x_g_1_2_sd3, x_g_1_2_sd4, x_g_1_2_sd5)
end_time = time.time()

print(f'Expected excess risk of SGLD is {expected_excess_risk}')
print(f'Value of SGLD is {theta_SGLD}')
print(f'耗时: {(end_time - start_time) / 5}秒')

476476
Expected excess risk of SGLD is 9.746890057354103e-05
Value of SGLD is 9.96584007337692
耗时: 131.36129717826844秒


In [182]:
## SGHMC

Model099_SGHMC_sd1 = SGHMC(beta, lam_c, lr, gamma, 0.99, seed_1, 10.200, 0.0001)
start_time = time.time()
theta_SGHMC, expected_excess_risk  = Model099_SGHMC_sd1.estimate(x_g_1_2_sd1, x_g_1_2_sd2, x_g_1_2_sd3, x_g_1_2_sd4, x_g_1_2_sd5)
end_time = time.time()

print(f'excess_risk of SGHMC is {expected_excess_risk}')
print(f'Value of SGHMC is {theta_SGHMC}')
print(f'耗时: {(end_time - start_time) / 5}秒')

218218
excess_risk of SGHMC is 9.826607416838469e-05
Value of SGHMC is 9.966151155302537
耗时: 55.3634072303772秒
